In [2]:
import openai, time, logging, os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve API key from environment variable
api_key = os.getenv("OPENAI_API_KEY")

# Set the API key for the openai module
openai.api_key = api_key

In [ ]:
model='gpt-4-turbo'

# Hardcoded assistant and thread IDs
assistant_id = 'asst_YAk6fYV9PvRR5PFnL4sn6quD'
thread_id = 'thread_ALbVkq1aTsobawtiG3NC1SvA'

In [3]:

model='ft:gpt-3.5-turbo-0125:formal-blocks:annotations-erc20:9UETgf4X'

# solidity solc vector store with eip md files
vector_store_id = 'vs_33UHiwTxfG0JszUC0lE7W2Dy'

# Create an assistant with the code interpreter tool
assistant = openai.beta.assistants.create(
  name="ERC20",
  instructions="Given an ERC token solidity code as input, return the exact same contract with the function annotations in the format accepted by the solc-verify smart contract verifier, just this, nothing more.",
  tools=[{"type": "file_search"}],
  model=model,
  tool_resources={
    "file_search": {
      "vector_store_ids": [vector_store_id]
    }
  }
)

print(assistant.id)

asst_GQKKwPlYuTjeg2X54lEsH0qY


In [ ]:

# Create a thread
thread = openai.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "You will help to generate formal specifications for smart contracts on the Ethereum network, written in Solidity and which will be verified in the solc-verify verifier, using exactly the solc-verify annotatation language syntax."
        }
    ]
)

print(thread.id)


In [ ]:
# Create a Message
message = '''
    Enter the contract here
'''
message = openai.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=message
)

In [ ]:
# Run the assistant
run = openai.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
    instructions="Given a ERC20 token solidity code as input, return the exact same contract with the functions annotations in the format accepted by the solc-verify smart contract verifier, just this, nothing more.",
)

def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
    """

    Wait for the run to complete, print the elapsed time, and save the response to a file.
    :param client: The OpenAI client.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if run.completed_at:
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime(
                    "%H:%M:%S", time.gmtime(elapsed_time)
                )
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")
                # Get messages here once Run is completed!
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                last_message = messages.data[0]
                response = last_message.content[0].text.value
                # print(f"Assistant Response: {response}")
                # save the response to a incrementing file with the user message at the top
                with open(f"response_{run_id}.txt", "w") as f:
                    f.write(f"User Message: {last_message.content[0].text.value}\n\n")
                    f.write(f"Assistant Response: {response}")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)

In [ ]:
wait_for_run_completion(client=openai, thread_id=thread_id, run_id=run.id)

#run_steps = client.beta.threads.runs.steps.list(thread_id=thread_id, run_id=run.id)
#print(f"Steps---> {run_steps.data[0]}")